In [1]:
import spikeinterface.full as si
print(f"SpikeInterface version: {si.__version__}")
from probeinterface import get_probe
from probeinterface.plotting import plot_probe

import os
import matplotlib
import numpy as np
from pathlib import Path
import ipywidgets as widgets
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter("ignore")

SpikeInterface version: 0.100.6


In [2]:
kilosort_25_path = '/home/bple/projects/Kilosort-2.5.2/'
irc_path = '/home/bple/projects/ironclust/'
si.Kilosort2_5Sorter.set_kilosort2_5_path(kilosort_25_path)
si.IronClustSorter.set_ironclust_path(irc_path)
si.installed_sorters()

Setting KILOSORT2_5_PATH environment variable for subprocess calls to: /home/bple/projects/Kilosort-2.5.2
Setting IRONCLUST_PATH environment variable for subprocess calls to: /home/bple/projects/ironclust


['ironclust',
 'kilosort2_5',
 'kilosort4',
 'mountainsort5',
 'spykingcircus',
 'spykingcircus2',
 'tridesclous2']

In [3]:
# method = 'kilosort4'
# dker = True
# params = si.get_default_sorter_params(method)
# params['Th_universal'] = 5
# params['Th_learned'] = 4
# params['min_template_size'] = 20
# params['nblocks'] = 0
# params['skip_kilosort_preprocessing'] = True

In [4]:
# method = 'kilosort2_5'
# dker = True
# params = si.get_default_sorter_params(method)
# params['nblocks'] = 0

In [5]:
# si.get_default_sorter_params(method)

In [6]:
method = 'mountainsort5'
dker = False
params = si.get_default_sorter_params(method)
params['detect_threshold'] = 4.0
params['detect_time_radius_msec'] = 1.5
params['filter'] = False
params['scheme2_training_duration_sec'] = 400
params['npca_per_subdivision'] = 8

In [7]:
# method = 'spykingcircus'
# dker = True
# params = si.get_default_sorter_params(method)
# params['filter'] = False
# params['detect_threshold'] = 5

In [8]:
# method = 'waveclus'
# dker = True
# params = si.get_default_sorter_params(method)

In [9]:
## Dimensions must match error
# method = 'ironclust'
# dker = True
# params = si.get_default_sorter_params(method)
# params['whiten'] = True
# params['filter'] = False
# params['freq_max'] = 6000

In [10]:
base_folder = Path('.')
oe_folder = Path(base_folder / './C49_2024-05-15_10-50-44_ar-nat8b')

sort_folder = Path(base_folder / 'sorting-data')
preprocess_folder = Path(sort_folder / 'preprocess')
sort_output_folder = Path(sort_folder / f'{method}-results')
waveform_folder = Path(sort_folder / "waveforms")
phy_output_folder = Path(sort_folder / f'phy-{method}')

In [11]:
n_cpus = os.cpu_count()
n_jobs = n_cpus - 4
job_kwargs = dict(n_jobs=n_jobs, progress_bar=True)
si.set_global_job_kwargs(**job_kwargs)

In [12]:
full_raw_rec = si.read_openephys(oe_folder)
full_raw_rec

OpenEphysBinaryRecordingExtractor: 40 channels - 30.0kHz - 1 segments - 82,394,880 samples 
                                   2,746.50s (45.77 minutes) - int16 dtype - 6.14 GiB

In [13]:
# Import premade probe
manufacturer = 'neuronexus'
model = 'A4x8-5mm-100-400-703'
probe = get_probe(manufacturer, model)
probe.wiring_to_device('ASSY-116>RHD2132')

In [14]:
raw_rec = full_raw_rec.set_probe(probe)
recording_f = si.bandpass_filter(raw_rec, freq_min=300, freq_max=6000)
recording_cmr = si.common_reference(recording_f, reference='global', operator='median')

In [15]:
# %matplotlib ipympl
# w = si.plot_traces({"raw": raw_rec, "filtered": recording_f, "common": recording_cmr}, mode='map',
#                    time_range=[10, 10.1], backend="ipywidgets")

In [16]:
if preprocess_folder.is_dir():
    recording_saved = si.load_extractor(preprocess_folder)
else:
    recording_saved = recording_cmr.save(folder=preprocess_folder, overwrite=False)
    print(f'Saved channels ids:\n{recording_saved.get_channel_ids()}')

In [17]:
sorting_data = si.run_sorter(sorter_name=method,
                             recording=recording_saved,
                             remove_existing_folder=True,
                             output_folder=sort_output_folder,
                             docker_image=dker,
                             verbose=True,
                             **params)

whitening
write_binary_recording with n_jobs = 8 and chunk_size = 30000


write_binary_recording:   0%|          | 0/2747 [00:00<?, ?it/s]

Using training recording of duration 400 sec with the sampling mode uniform
*** MS5 Elapsed time for SCHEME2 get_sampled_recording_for_training: 1.512 seconds ***
Running phase 1 sorting
Number of channels: 32
Number of timepoints: 12000000
Sampling frequency: 30000.0 Hz
Channel 0: [800.   0.]
Channel 1: [800. 200.]
Channel 2: [800. 400.]
Channel 3: [800. 600.]
Channel 4: [1200.    0.]
Channel 5: [1200.  200.]
Channel 6: [1200.  400.]
Channel 7: [1200.  600.]
Channel 8: [1200.  700.]
Channel 9: [1200.  500.]
Channel 10: [1200.  300.]
Channel 11: [1200.  100.]
Channel 12: [800. 700.]
Channel 13: [800. 500.]
Channel 14: [800. 300.]
Channel 15: [800. 100.]
Channel 16: [400. 700.]
Channel 17: [400. 500.]
Channel 18: [400. 300.]
Channel 19: [400. 100.]
Channel 20: [  0. 700.]
Channel 21: [  0. 500.]
Channel 22: [  0. 300.]
Channel 23: [  0. 100.]
Channel 24: [0. 0.]
Channel 25: [  0. 200.]
Channel 26: [  0. 400.]
Channel 27: [  0. 600.]
Channel 28: [400.   0.]
Channel 29: [400. 200.]
Channe

In [18]:
sorting_data.save(folder = sort_output_folder, overwrite=True)

NumpyFolderSorting: 41 units - 1 segments - 30.0kHz

In [19]:
we = si.extract_waveforms(recording_saved, sorting_data,
                          folder = waveform_folder,
                          sparse = False, overwrite=True)
print(we)
sparsity = si.compute_sparsity(we, method='radius', radius_um=80.0)

extract waveforms memmap multi buffer:   0%|          | 0/2747 [00:00<?, ?it/s]

WaveformExtractor: 32 channels - 41 units - 1 segments
  before:30 after:60 n_per_units:500


In [20]:
we = si.extract_waveforms(recording_saved, sorting_data,
                          folder = waveform_folder,
                          sparsity = sparsity,
                          overwrite=True)
print(we)

extract waveforms memmap multi buffer:   0%|          | 0/2747 [00:00<?, ?it/s]

WaveformExtractor: 32 channels - 41 units - 1 segments
  before:30 after:60 n_per_units:500 - sparse


In [21]:
pc = si.compute_principal_components(we, mode='by_channel_global', n_components=3, load_if_exists=False)

Fitting PCA:   0%|          | 0/41 [00:00<?, ?it/s]

Projecting waveforms:   0%|          | 0/41 [00:00<?, ?it/s]

In [22]:
si.export_to_phy(we,
                 output_folder=phy_output_folder,
                 compute_amplitudes=True,
                 compute_pc_features=True,
                 remove_if_exists=True,
                 copy_binary=False,
                 )

extract amplitudes:   0%|          | 0/2747 [00:00<?, ?it/s]

extract PCs:   0%|          | 0/2747 [00:00<?, ?it/s]

Run:
phy template-gui  /home/bple/projects/sorting-data/phy-mountainsort5/params.py
